In [ ]:
import tweepy
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax

# Set your Twitter API credentials
consumer_key = 'YOUR_CONSUMER_KEY'
consumer_secret = 'YOUR_CONSUMER_SECRET'
access_token = 'YOUR_ACCESS_TOKEN'
access_token_secret = 'YOUR_ACCESS_TOKEN_SECRET'

# Set the target username and number of recent tweets to fetch
username = 'target_username'
num_tweets = 15

# Set up Twitter API authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

# Fetch recent tweets of the user
tweets = api.user_timeline(screen_name=username, count=num_tweets, tweet_mode='extended')

# Load the RoBERTa sentiment analysis model and tokenizer
roberta_model_name = "cardiffnlp/twitter-roberta-base-sentiment"
model = AutoModelForSequenceClassification.from_pretrained(roberta_model_name)
tokenizer = AutoTokenizer.from_pretrained(roberta_model_name)
labels = ['Negative', 'Neutral', 'Positive']

# Analyze sentiment of each fetched tweet
for tweet in tweets:
    tweet_text = tweet.full_text
    tweet_words = []

    # Preprocess the tweet text
    for word in tweet_text.split(' '):
        if word.startswith('@') and len(word) > 1:
            word = '@user'
        elif word.startswith('http'):
            word = "http"
        tweet_words.append(word)
    
    tweet_proc = " ".join(tweet_words)
    
    encoded_tweet = tokenizer(tweet_proc, return_tensors='pt')
    output = model(**encoded_tweet)
    
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    
    max_score_index = scores.argmax()
    max_label = labels[max_score_index]
    
    print("Tweet:", tweet_text)
    print("Sentiment:", max_label)
    print()